APAN 5400 Final Project

In [ ]:
# Packages
import console
import psycopg
import requests
from datetime import datetime, timedelta

In [2]:
# Muted this code for now, because we don't need to use it every time 

# import requests
# from pymongo import MongoClient
# from datetime import datetime, timedelta

# client = MongoClient('localhost', 27017)
# db = client.apan5400
# news_collection = db['news_data']

# def fetch_and_store_news(api_key, companies):
#     for company in companies:
#         period_months = 1  # longest time period we can get with free ap 
#         start_date = datetime.today() - timedelta(days=30 * period_months)
#         start_date_str = start_date.strftime('%Y-%m-%d')

#         url = f"https://newsapi.org/v2/everything?q={company}&from={start_date_str}&sortBy=popularity&apiKey={api_key}"
#         response = requests.get(url)
#         if response.ok:
#             news_data = response.json()['articles']
#             for article in news_data:
#                 # make sure title exist and is not null
#                 if article.get('title') and "Removed" not in article['title']:
#                     # make sure exist
#                     article['author'] = article.get('author', "Author information not available")
#                     article['description'] = article.get('description', "Description not available")
#                     article['url'] = article.get('url', "URL not available")
#                     article['publishedAt'] = article.get('publishedAt', "publishedAt not available")
#                     # store news into MongoDB
#                     news_collection.update_one({'url': article['url']}, {'$set': article}, upsert=True)
#         else:
#             print(f"Failed to fetch news for {company}")

# api_key = '3bbf923e85b640a0818dae15b608d4f5'  # our API key 
# companies = [
#     "Apple", "Google", "Microsoft", "Amazon", "Facebook",
#     "Tesla", "Alibaba", "Samsung", "Intel", "IBM",
#     "Oracle", "Cisco", "Visa", "MasterCard", "PayPal",
#     "Adobe", "Salesforce", "Netflix", "Walmart", "Disney",
#     "Nike", "Coca-Cola", "Pepsi", "McDonald's", "Starbucks",
#     "General Motors", "Ford", "AT&T", "Verizon", "ExxonMobil",
#     "Chevron", "Goldman Sachs", "Morgan Stanley", "JP Morgan Chase", "Citigroup",
#     "Bank of America", "HSBC", "Barclays", "Royal Dutch Shell", "BP",
#     "Siemens", "Bayer", "Airbus", "Boeing", "Philips",
#     "Sony", "Panasonic", "LG", "Huawei", "Xiaomi"
# ] # insert as many as you want 

# fetch_and_store_news(api_key, companies)

In [3]:
# Packages
from flask import Flask, jsonify, request, Response
from flask import Flask, session, redirect, url_for
from pymongo import MongoClient
from bson import json_util
from flask import Flask, jsonify, request, render_template
import json
from flask_session import Session 

app = Flask(__name__)

app.config['SESSION_TYPE'] = 'filesystem'
Session(app)

# connecting to the mongoDB 
client = MongoClient('localhost', 27017)
db = client.apan5400
news_collection = db['news_data']

@app.route('/')
def index():
    return render_template('index.html')

In [4]:
# connecting to the SQL database 
def get_db_connection():
    conn = psycopg.connect(
        host="localhost",
        port='5432',
        dbname="APAN5400",
        user="postgres",
        password="123")
    return conn

In [5]:
# @app.route('/login', methods=['GET', 'POST'])
# def login():
#     if request.method == 'POST':
#         username = request.form['username']
#         conn = get_db_connection()
#         cursor = conn.cursor()
#         cursor.execute("SELECT * FROM users WHERE username = %s", (username,))
#         if cursor.fetchone():
#             session['username'] = username
#             return redirect(url_for('search'))
#         else:
#             return "Username not found", 404
#     return render_template('login.html')

# login program
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        data = request.get_json()
        username = data.get('username')
        print(username)
        
        if not username:
            return jsonify({"error": "Username is required"}), 400

        conn = get_db_connection()
        cursor = conn.cursor()

        try:
            cursor.execute("SELECT user_id FROM users WHERE username = %s", (username,))
            user_id_result = cursor.fetchone()
            if user_id_result:
                user_id = user_id_result[0]
                print(user_id)

                session['username'] = username
                return jsonify({"success": True}), 200
            else:
                return jsonify({"error": "User does not exist"}), 404

        except Exception as e: 
            conn.rollback()
            return jsonify({"error": str(e)}), 500
        finally:
            cursor.close()
            conn.close()

    return render_template('login.html')

In [6]:
@app.route('/info')
def info():
#     if 'username' not in session:
#         return redirect(url_for('login'))
    return render_template('info.html')

In [7]:
# call the news_api key to obtain the news 
from datetime import datetime, timedelta

def get_news_data(search_keyword, period='1'):
    period_months = int(period)
    start_date = datetime.today() - timedelta(days=30 * period_months)
    start_date_str = start_date.strftime('%Y-%m-%d')

    api_key = '3bbf923e85b640a0818dae15b608d4f5' 
    url = f"https://newsapi.org/v2/everything?q={search_keyword}&from={start_date_str}&sortBy=popularity&apiKey={api_key}"
    response = requests.get(url)
    if response.ok:
        news_data = response.json()['articles']
        for article in news_data:
            if "Removed" not in article['title']:
                # make sure it exist
                article['author'] = article.get('author', "Author information not available")
                article['description'] = article.get('description', "Description not available")
                article['url'] = article.get('url', "URL not available")
                article['publishedAt'] = article.get('publishedAt', "publishedAt not available")
                # store news into MongoDB
                news_collection.update_one({'url': article['url']}, {'$set': article}, upsert=True)
    else:
        print("Failed to fetch news data")
        return []

    # draw all the news from the database
    all_news = list(news_collection.find({'title': {'$regex': search_keyword, '$options': 'i'}}))
    return all_news  

In [8]:
@app.route('/search')
def search():
        if 'username' not in session:
            # return redirect(url_for('login'))
            print("failed")
        return render_template('search.html')

# filter by publisher
@app.route('/filter_news_by_publisher', methods=['POST'])
def filter_news_by_publisher():
    publisher = request.form.get('publisher')
    company_name = request.form.get('company_name')
    period = request.form.get('period', '1')

    if company_name:
        news_data = get_news_data(company_name, period)

        if news_data:
            filtered_news = [news for news in news_data if news.get('source', {}).get('name') == publisher]
            ids_to_find = [news['_id'] for news in filtered_news]
            query = {'_id': {'$in': ids_to_find}}

            cursor = news_collection.find(query)
            list_cursor = list(cursor)
            response = jsonify(json.loads(json_util.dumps(list_cursor)))

            return response, 200
        else:
            return jsonify({"error": "No stock data found"}), 404
    else:
        return jsonify({"error": "Stock code is required"}), 400

# save search_history and insert into SQL database 
@app.route('/save_search_history', methods=['POST'])
def save_search_history():
    if 'username' in session:
        username = session['username']
        company_name = request.form.get('company_name')
        period = request.form.get('period')
        search_date = datetime.now().strftime('%Y-%m-%d')
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO search_history (username, search_keyword, search_date, period) VALUES (%s, %s, %s, %s)",
            (username, company_name, search_date, period)
        )
        conn.commit()
        cursor.close()
        conn.close()
        return 'History saved', 200
    else:
        return 'User not logged in', 403

In [9]:
# Obtain stock data from yahoo finance 
import yfinance as yf

def get_stock_data(company_code, period='1'):
    stock = yf.Ticker(company_code)
    hist = stock.history(period=f"{period}mo")
    return {
        "company": company_code,
        "current_price": f"${round(float(hist.iloc[-1]['Close']),2)}",
        "high_price": f"${round(float(hist['High'].max()),2)}",
        "low_price": f"${round(float(hist['Low'].min()),2)}",
        "open_price": f"${round(float(hist.iloc[0]['Open']),2)}",
        "volume": int(hist['Volume'].sum()),
        "dividends": round(float(hist['Dividends'].sum()),2)
    }

@app.route('/get_stock_data')
def fetch_stock_data():
    company_code = request.args.get('company_code','AAPL')
    period = request.args.get('period', '1')
    print(company_code)
    print(period)

    data = get_stock_data(company_code, period)
    return jsonify(data)

In [10]:
# Packages
import matplotlib.pyplot as plt
from flask import send_file
from io import BytesIO
import yfinance as yf

@app.route('/plot_stock/<string:stock_code>/<int:period>')
def plot_stock(stock_code, period):
    # Fetching the complete DataFrame just for plotting
    stock = yf.Ticker(stock_code)
    hist = stock.history(period=f"{period}mo")

# plot the data
    if 'Close' not in hist.columns:
        return "Error: 'Close' column not found in the data", 400

    plt.figure(figsize=(10, 5))
    plt.plot(hist.index, hist['Close'], label='Close Price')  
    plt.title(f'{stock_code.upper()} Stock Closing Prices Over {period} Month(s)')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.grid(True)
    plt.legend()

    img = BytesIO()
    plt.savefig(img, format='png')
    plt.close()
    img.seek(0)

    return send_file(img, mimetype='image/png')

In [ ]:
app.run(host='localhost', port=5006)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5006
Press CTRL+C to quit
127.0.0.1 - - [25/Apr/2024 16:46:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:46:39] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:46:39] "GET /static/js/script.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:46:41] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:46:41] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:46:41] "GET /static/js/script_login.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:46:42] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:46:42] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:46:42] "GET /static/js/script.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:46:46] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:46:47] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:46:47] "GET /static/js/script.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:46:50] "GE

Chenyu Mao
2


127.0.0.1 - - [25/Apr/2024 16:47:07] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:47:07] "GET /static/js/script_login.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:47:07] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:47:38] "POST /login HTTP/1.1" 404 -


123


...

127.0.0.1 - - [25/Apr/2024 16:49:48] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:49:48] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:49:49] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:49:49] "GET /static/js/script.js HTTP/1.1" 304 -


Chenyu Mao
2


127.0.0.1 - - [25/Apr/2024 16:50:00] "POST /save_search_history HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:01] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:28] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:29] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:31] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:33] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:35] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:36] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:42] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:50:51] "POST /filter_news_by_publisher HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:51:03] "GET /info HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:51:03] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 

AAPL
1


127.0.0.1 - - [25/Apr/2024 16:51:38] "GET /get_stock_data?company_code=AAPL&period=1 HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:51:39] "GET /plot_stock/AAPL/1 HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:52:04] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 16:52:04] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2024 16:52:04] "GET /static/js/script.js HTTP/1.1" 304 -
